#### Installations

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn-extra

Note: you may need to restart the kernel to use updated packages.


### Helper Functions: Algorithms & Clustering

**Algorithms**

DONE: add a "mode" argument to each algorithm that, if mode = 1 the cluster labelling is output and if mode = 0 the silhouette coefficient is output.

In [5]:
# Code for KMeans

import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from sklearn.metrics import silhouette_score

"""
Performs KMeans clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def kmeans_clustering(samples,mode,  n_clusters=2, max_iter=300):
    """
    Perform KMeans clustering on the input samples
    
    Parameters:
        samples: array-like, shape (n_samples, n_features)
        n_clusters: int, number of clusters (default=2)
        max_iter: int, maximum iterations (default=300)
    
    Returns:
        silhouette_coef: silhouette coefficient score
    """
    k_means = KMeans(n_clusters=n_clusters, max_iter=max_iter)
    k_means.fit(samples)
    if mode == 0:
        try:
            silhouette_coef = silhouette_score(samples, k_means.labels_, metric='euclidean')
        except ValueError:
            silhouette_coef = 0  # Assigning lowest score if clustering fails
        return silhouette_coef
    if mode == 1:
        return k_means.labels_

In [6]:
# EM Clustering Code

from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


"""
Performs EM clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def em_clustering(selected_features, mode, n_clusters=2):
    """
    Perform EM Clustering on selected features and return silhouette score.
        
    Returns:
    --------
    float
        Silhouette score of the clustering (-1 if clustering fails)
    """
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the EM model
    em_model = GaussianMixture(
        n_components=n_clusters,
        random_state=0, #THOUGHTS: We can improve this later to have an array of seeds to select from to observe variations
        n_init=10  # Multiple initializations to avoid local optima
    )
    
   
    try:
        # Fit the model and get cluster assignments
        em_model.fit(X_scaled)
        labels = em_model.predict(X_scaled)
        
        # Calculate silhouette score
        silhouette_coef = silhouette_score(X_scaled, labels)
    except Exception as e:
        #print(f"Clustering failed: {str(e)}")
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

In [7]:
# DBSCAN Detection method: 
# I put 'optimization part' in 'DBSCAN_Optimization_Code.ipynb' file. 
# We can use optimization after initial run to do a comparison and analysis in our paper to show improvements.

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# DONE: keep -1 --> they will be its own cluster
"""
Performs DBSCAN clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def dbscan_clustering(selected_features, mode, eps=0.5, min_samples=5):
    """
    Perform DBSCAN clustering on selected features
    
    Parameters:
    selected_features : pandas DataFrame
        The features selected for clustering
    eps : float
        The maximum distance between two samples for them to be considered neighbors
    min_samples : int
        The number of samples in a neighborhood for a point to be considered a core point
        
    Returns:
    float : silhouette coefficient
    dict : additional clustering information
    """

    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize and fit DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)
    
    # Get number of clusters (excluding noise points which are labeled -1, K Medoids does not have noise points)
    n_clusters = len(set(labels))

    if -1 in labels:
        labels[labels == -1] = n_clusters - 1
    
    # calculate silhouette score if more than one cluster and  noise points
    if n_clusters > 1:
        silhouette_coef = silhouette_score(X_scaled, labels)
    else:
        silhouette_coef = 0  # Assign lowest score if clustering fails

    
    # NOTE: -- Uncomment when we analyze and optimize ---- Additional clustering information
    # info = {
    #     'n_clusters': n_clusters,
    #     'n_noise': list(labels).count(-1),
    #     'labels': labels,
    #     'cluster_sizes': pd.Series(labels).value_counts().to_dict()
    # }
    
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels


In [10]:
# Code for K Medoids
import numpy as np
import pandas as pd

from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

"""
Performs KMediods clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def modified_kmedoids_clustering(selected_features, mode, n_clusters=2):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

     # Initialize and fit the K-Medoids model
    kmedoids = KMedoids(n_clusters=n_clusters, method='alternate', max_iter=1500)
    

    # Calculate silhouette score
    try:
        labels = kmedoids.fit_predict(X_scaled)
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

In [8]:
# Code for K Medoids
import numpy as np
import pandas as pd

from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

"""
Performs KMediods clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def kmedoids_clustering(selected_features, mode, n_clusters=2):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

     # Initialize and fit the K-Medoids model
    kmedoids = KMedoids(n_clusters=n_clusters, method='pam', max_iter=1500, random_state=0)
    

    # Calculate silhouette score
    try:
        labels = kmedoids.fit_predict(X_scaled)
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assigning lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        return labels

In [11]:
# Code for Mean Shift
import numpy as np
import pandas as pd

from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

"""
Performs Mean Shift clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def modified_meanshift_clustering(selected_features, mode, quantile=0.2, n_samples=500):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Estimate optimal bandwidth
    bandwidth = estimate_bandwidth(X_scaled, quantile=quantile, n_samples=n_samples)
    if bandwidth <= 0:
        bandwidth = 1.0  # Fallback in case of extremely small bandwidth
        
    # Initialize and fit the Mean Shift model
    meanshift = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    # Calculate silhouette score
    try:
        # print("in try")
        labels = meanshift.fit_predict(X_scaled)
        n_clusters = len(set(labels))

        # if -1 in labels:
        #     labels[labels == -1] = n_clusters - 1
        # Check the number of clusters determined 
        #n_clusters = len(np.unique(labels))
        #print(f"Number of clusters found: {n_clusters}")
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        # print("in except")
        silhouette_coef = 0  # Assign lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        
        return labels

In [15]:
# Codes for Mean Shift
import numpy as np
import pandas as pd

from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

"""
Performs Mean Shift clustering using the data from selected_features.
If mode = 0, the silhouette score of the clustering is returned.
If mode = 1, the labels of the clustering is returned.
"""
def meanshift_clustering(selected_features, mode, bandwidth=None):
    # Filter the selected features
    X = selected_features
    
    # Standardize selected features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize and fit the Mean Shift model
    meanshift = MeanShift(bandwidth=bandwidth)
    
    # Calculate silhouette score
    try:
        labels = meanshift.fit_predict(X_scaled)
        # Check the number of clusters determined 
        n_clusters = len(np.unique(labels))
        #print(f"Number of clusters found: {n_clusters}")
        silhouette_coef = silhouette_score(X_scaled, labels)
    except ValueError:
        silhouette_coef = 0  # Assign lowest score if clustering fails
    if mode == 0:
        return silhouette_coef
    if mode == 1:
        
        return labels

In [12]:
"""
Converts the binary value of state (which represents features selected) 
to a string output res and returns.
"""
def bin_to_features(state):
  state_bin = bin(state)
  #print(state_bin)
  state_bin_arr = np.array([b for b in state_bin[2:]])
  # pad with zeros
  diff = len(FEATURES) - len(state_bin_arr)
  padded_arr = np.insert(state_bin_arr, 0, ['0' for i in range(diff)])
  #(padded_arr)
  # identify which indexes are 1
  idx = (np.where(padded_arr == '1')[0]).tolist()
  #print(idx)
  # select feature headings
  selected_features = original_features.iloc[:,idx]
  features = selected_features.columns.tolist()
  res = f"Features Used: {features}"
  return res


### Main Code

In [153]:
# features = np.array(['action', 'availability', 'device_type:1', 'direction',
#        'event_type', 'interface_status', 'patch_description',
#        'patch_status', 'severity:1', 'traffic_direction'])

# Replace feature list input with selected features
# features = np.array(['action', 'behavior_name', 'event_id:4', 'exploit_available',
#  'file_signature_status', 'network_interface', 'patch_name', 'severity',
#  'tcp_flags', 'vulnerability_solution']) # features for generated data complete

#features = np.array(['avg_bytes_sent','avg_bytes_received','avg_packets_transferred','avg_flow_duration','recent_tcp_flags','recent_protocol','avg_cpu_usage','avg_memory_usage','avg_disk_usage','avg_uptime']) # features for generated data

features = np.array(['service', 'id.resp_p', 'proto', 'conn_state', 'history', 'resp_pkts', 'orig_pkts', 'orig_bytes', 'resp_ip_bytes', 'orig_ip_bytes']) # modified features for RW data

features = features.tolist()
print(features)

['service', 'id.resp_p', 'proto', 'conn_state', 'history', 'resp_pkts', 'orig_pkts', 'orig_bytes', 'resp_ip_bytes', 'orig_ip_bytes']


In [ ]:
import pandas as pd
import numpy as np

# FEATURES = {0: 'avg_bytes_sent', 1: 'avg_bytes_received', 2: 'avg_packets_transferred', 
#   3: 'avg_flow_duration', 4: 'recent_tcp_flags', 5: 'recent_protocol', 6: 'avg_cpu_usage', 
#   7: 'avg_memory_usage', 8: 'avg_disk_usage', 9: 'avg_uptime'}
# features = list(FEATURES.values())
FEATURES = {k:str(v) for k,v in zip(range(len(features)), features) }
#print(FEATURES)
data = pd.read_csv("../data/real-world/data-cleaning/cleaned_RW21.csv") # UPDATE NEW DATA HERE
#data = pd.read_csv("../data/generated/data-cleaning/cleaned_source_data.csv")
#data = pd.read_csv("../data/generated/processed/feature_selection_quantitative.csv")

#print(data.head(10))


ALGORITHMS = {0: 'DBSCAN Clustering', 1: 'Mean Shift', 2: 'K-Mediods', 3: 'EM Clustering', 4: 'K-Means'}
NUM_ALG = len(ALGORITHMS)
original_features = data[features].copy(deep = True)
print(original_features.head(10))
uid = "uid"
ips = data[uid]
#print(original_features.columns)
#print(ips.head(10))

def algorithm_prep(state, action, mode):
  # convert state to binary
  state_bin = bin(state)
  #print(state_bin)
  state_bin_arr = np.array([b for b in state_bin[2:]])
  # pad with zeros
  diff = len(FEATURES) - len(state_bin_arr)
  padded_arr = np.insert(state_bin_arr, 0, ['0' for i in range(diff)])
  #(padded_arr)
  # identify which indexes are 1
  idx = (np.where(padded_arr == '1')[0]).tolist()
  #print(idx)
  # select feature headings
  selected_features = original_features.iloc[:,idx]
  #print(selected_features.head(10))
  # select algorithm
  # algo = action
  # prep correct data - done
  
  # call algorithm function
  out = None
  #print('algorithm:',ALGORITHMS[action])

  # if mode = 0, output is the silhouette coefficient
  # if mode = 1, output is the cluster labelling
  match action:   
    case 0:
      #print('algorithm:',ALGORITHMS[action])
      out = dbscan_clustering(selected_features, mode)
    case 1: 
      #print('algorithm:',ALGORITHMS[action])
      out = modified_meanshift_clustering(selected_features, mode)
    case 2:
      #print('algorithm:',ALGORITHMS[action])
      out = modified_kmedoids_clustering(selected_features, mode)
    case 3: 
      out = em_clustering(selected_features, mode)
    case 4:
      out = kmeans_clustering(selected_features, mode)
      
  # return silhouette from algorithm function
  return out

   service  id.resp_p  proto  conn_state  history  resp_pkts  orig_pkts  \
0        0         53      0           0        0          0          1   
1        0         53      0           1        1          2          2   
2        0         53      0           0        0          0          2   
3        0         53      0           1        1          2          2   
4        0         53      0           1        1          2          3   
5        0         53      0           1        1          2          2   
6        0         53      0           0        0          0          2   
7        0         53      0           1        1          2          2   
8        0         53      0           0        0          0          2   
9        1        123      0           0        0          0          1   

   orig_bytes  resp_ip_bytes  orig_ip_bytes  
0           0              0             67  
1           1            146            146  
2           2              0        

##### Sampling (Test Only)

In [ ]:
# # idx = (np.where(percentages <= 0.1)[0]).tolist()
# idx = []
# anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
# print(len(anomalies))
perc_values = labelled_data['cluster'].value_counts(ascending=True)
print(perc_values)
print("flipped:",perc_values.values)
print(percentages)
idx = (np.where(percentages <= 0.1)[0]).tolist()
print(idx)
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
print(len(anomalies))
perc_values = labelled_data['cluster'].value_counts().values

In [19]:
label_test = algorithm_prep(256, 4,1)
labelled_data = data.copy()
labelled_data['cluster'] = label_test
labelled_data.head(10)

num_clusters = labelled_data['cluster'].nunique()
vals = labelled_data['cluster'].value_counts().values
print(labelled_data.shape[0])
vals = vals / labelled_data.shape[0]
idx = (np.where(vals <= 0.1)[0]).tolist()
print(idx)
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
#print(anomalies)
for i in anomalies['source_ip']:
    print(f"IP {i} is a potential anomaly")

297
[1]
IP 192.168.0.248 is a potential anomaly
IP 192.168.0.78 is a potential anomaly


#### Reinforcement Learning

In [155]:
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2018 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
# 1024 configurations of the 10 features --> 2^10
# 5 algorithms
num_configs = 2 ** len(FEATURES)
R = ql.matrix(ql.zeros([num_configs,5]))

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([num_configs,5]))

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
# 1) DONE: decide if starting state is random or a specific state
agent_s_state = 1

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    # 2) DONE: we should check Q, not R because R is never modified
    current_state_row = Q[state,]
    # 3) DONE: this should pick valid actions based on what we have not visited
    possible_act = ql.where(current_state_row == 0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1)[0])
    if(sum(PossibleAction)<=0):
        next_action = int(np.random.choice(NUM_ALG,1)[0])
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

# A version of Bellman's equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1)[0])
    else:
        Max_State = int(Max_State[0])

    # 5) DONE: we think this is a typo and action/Max_State should be switched. 
    # MaxValue = Q[action, Max_State]
    MaxValue = Q[Max_State, action]

    # 6) DONE: call function to run ML algorithm using the value of action. this will
    # run the algorithm using the features from current_state, create clusters,
    # and calculate the silhouette value.
    silhouette_co = algorithm_prep(current_state, action, 0) 
    
    # Bellman's MDP based Q function
    # 7) DONE: instead of getting a value from R, we add the silhouette value to gamma * MaxValue
    # Q[current_state, action] = R[current_state, action] + gamma * MaxValue
    Q[current_state, action] = silhouette_co + gamma * MaxValue


# Rewarding Q matrix
reward(agent_s_state,action,gamma)


# Leraning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(3000):
    print("Iteration:", i)
    # select a random new state (configuration of features)
    current_state = ql.random.randint(1, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    print("Algorithm:", ALGORITHMS[action])
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

# DONE: get maximum value from Q-Learning Matrix
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("\nmax value located at",max_location)
max_config = max_location[0][0]
max_algorithm = max_location[1][0]
final_feats = bin_to_features(max_config)
print(f"\nUsing algorithm {ALGORITHMS[max_algorithm]} and {final_feats}, max value is:",normed_Q[max_config,max_algorithm])
#DONE: print(f"Selected features:")

# DONE: get final cluster labels
cluster_labels = algorithm_prep(max_config, max_algorithm, 1)

# DONE: match data in clusters to IP addresses
labelled_data = data.copy()
labelled_data['cluster'] = cluster_labels

# DONE: return what IPs are likely anomalous
# see what clusters have < 5% of the data
# get unique values in cluster column
num_clusters = labelled_data['cluster'].nunique()

# for each unique value, get the count / len of data (aka percentage)
# num_clusters = labelled_data['cluster'].nunique()
cluster_array = labelled_data['cluster'].to_numpy()
perc_values = np.unique(cluster_array,return_counts = True)[-1]
percentages = perc_values / labelled_data.shape[0]

# keep cluster values with % < 5
idx = (np.where(percentages <= 0.1)[0]).tolist()
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
# output IPs within those selected clusters

for i in anomalies[uid]: # replace with unique ID here
    print(f"\nIP {i} is a potential anomaly")

Iteration: 0
Algorithm: Mean Shift
Iteration: 1
Algorithm: K-Means
Iteration: 2
Algorithm: EM Clustering
Iteration: 3
Algorithm: Mean Shift
Iteration: 4
Algorithm: DBSCAN Clustering
Iteration: 5
Algorithm: EM Clustering
Iteration: 6
Algorithm: EM Clustering
Iteration: 7
Algorithm: DBSCAN Clustering
Iteration: 8
Algorithm: DBSCAN Clustering
Iteration: 9
Algorithm: DBSCAN Clustering
Iteration: 10
Algorithm: K-Mediods


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(


Iteration: 11
Algorithm: Mean Shift
Iteration: 12
Algorithm: DBSCAN Clustering
Iteration: 13
Algorithm: K-Means
Iteration: 14
Algorithm: DBSCAN Clustering
Iteration: 15
Algorithm: DBSCAN Clustering
Iteration: 16
Algorithm: K-Mediods


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(


Iteration: 17
Algorithm: Mean Shift
Iteration: 18
Algorithm: EM Clustering
Iteration: 19
Algorithm: K-Mediods


...

Iteration: 2988
Algorithm: Mean Shift
Iteration: 2989
Algorithm: DBSCAN Clustering
Iteration: 2990
Algorithm: EM Clustering
Iteration: 2991
Algorithm: Mean Shift
Iteration: 2992
Algorithm: Mean Shift
Iteration: 2993
Algorithm: K-Mediods


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(


Iteration: 2994
Algorithm: K-Mediods


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(


Iteration: 2995
Algorithm: EM Clustering
Iteration: 2996
Algorithm: Mean Shift
Iteration: 2997
Algorithm: K-Means
Iteration: 2998
Algorithm: Mean Shift
Iteration: 2999
Algorithm: EM Clustering
Q  :
[[0.         0.         0.         0.         0.        ]
 [0.99876834 0.         0.         0.         0.99960697]
 [0.         0.         0.         0.         0.99954561]
 ...
 [0.96094217 0.         0.         0.89762083 0.        ]
 [0.         0.95349166 0.         0.89761949 0.        ]
 [0.96087236 0.95328551 0.         0.64346116 0.99949325]]
Normed Q :
[[ 0.          0.          0.          0.          0.        ]
 [55.50914702  0.          0.          0.         55.55575614]
 [ 0.          0.          0.          0.         55.55234595]
 ...
 [53.406859    0.          0.         49.88761123  0.        ]
 [ 0.         52.99277782  0.         49.88753648  0.        ]
 [53.40297928 52.98132045  0.         35.76202647 55.5494354 ]]

max value located at (array([717]), array([4]))

Usi

In [156]:
print(f"\nUsing algorithm {ALGORITHMS[max_algorithm]} and {final_feats}, max value is:",normed_Q[max_config,max_algorithm])
anomalies


Using algorithm K-Means and Features Used: ['service', 'proto', 'conn_state', 'orig_pkts', 'orig_bytes', 'orig_ip_bytes'], max value is: 100.0


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,detailed-label,cluster
2034,1538573216,CChUAn2y9k075ut2L4,0,41176,3,443,1,1,703,13,...,1,2472,4,6793,285885,7129,301794,1,1,1
3146,1538607284,CeXTE32bb03yltkRV,0,41184,3,443,1,1,879,18,...,4,5390,7,5895,248182,6147,257816,1,1,1
3285,1538636962,CJ0N5q2qFD0t1nCl3j,0,41186,3,443,1,1,910,19,...,5,2472,8,4485,189231,4702,200386,1,1,1


In [145]:
max_location

(array([128]), array([0]))

In [129]:
algorithm_prep(current_state, 5, 0)
# alg prep: dbscan 0.2s, meanshift 10-16s (modified: 1s), kmediods 0.6s, em clustering 0.3s, kmeans 0.1s


np.float64(0.5744217212371902)

In [126]:
algorithm_prep(current_state, action, 0)


/opt/homebrew/lib/python3.10/site-packages/sklearn_extra/cluster/_k_medoids.py:297: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


np.float64(0.5744217212371902)

selected_features

#### ignore

In [27]:
import numpy as np
# percentages = labelled_data['cluster'].value_counts().values
# percentages = percentages / labelled_data.shape[0]
# print(percentages)
# print(labelled_data['cluster'].value_counts())
cluster_array = labelled_data['cluster'].to_numpy()
perc_vals = np.unique(cluster_array,return_counts = True)[-1]
print(perc_vals)
percentages = perc_vals / labelled_data.shape[0]
print(percentages)
idx = (np.where(percentages <= 0.1)[0]).tolist()
anomalies = labelled_data.loc[labelled_data['cluster'].isin(idx)]
print(anomalies)

[296   1]
[0.996633 0.003367]
    device_name      source_ip  avg_bytes_sent  avg_bytes_received  \
142  Device-227  192.168.0.226          9973.5              4648.0   

     avg_packets_transferred  avg_flow_duration  recent_tcp_flags  \
142                   303.75            7694.75                 2   

     recent_protocol  avg_cpu_usage  avg_memory_usage  avg_disk_usage  \
142                1         29.545            58.905           58.96   

     avg_uptime  cluster  
142       426.0        1  


In [ ]:
#print(max_location)
np.where(normed_Q==normed_Q.max())
#normed_Q[normed_Q > 99.]
#max_location = np.where(normed_Q==normed_Q.max())

matrix([[ 99.99998773, 100.        ,  99.99998111,  99.9512457 ,
          99.9512457 ,  99.95124659,  99.99917226,  99.99991577,
          99.9991523 ,  99.99991297,  99.95122339,  99.95121864]])

In [101]:
normed_Q = Q/ql.max(Q)*100
max_location = np.where(normed_Q==normed_Q.max())
print("max value located at",max_location)
max_config = max_location[0][0]
max_algorithm = ALGORITHMS[max_location[1][0]]
print(f"Using algorithm {max_algorithm} and feature configuration {max_config}, max value is:",normed_Q[278,0])



max value located at (array([278]), array([0]))
278
Using algorithm K-Means and feature configuration 278, max value is: 100.0


### Performance Metrics

In [184]:
from sklearn.metrics import average_precision_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score

In [158]:
labels = data['label']
result_labels = pd.DataFrame(columns=["true", "pred"])
result_labels['uid'] = ips
result_labels['true'] = labels
#print(result_labels)
result_labels.loc[result_labels['uid'].isin(anomalies[uid]), 'pred'] = 1
result_labels.loc[~result_labels['uid'].isin(anomalies[uid]), 'pred'] = 0
print(result_labels.loc[result_labels['true'] == 1])


      true pred                 uid
20       1    0  CaFD7t4x11rNjJyXpe
22       1    0   C2gDegk8EvlOlAaJi
25       1    0  CY9Vox2T9RVyTvV5Zc
2034     1    1  CChUAn2y9k075ut2L4
2035     1    0  CPQreG2wGyBw1Fdqa2
2046     1    0  C5eeVN1ibcDUKaR0na
2048     1    0  CUpb0u1h3TX688LlAi
2049     1    0  CImwQs4If4Eyj29ZQ6
2050     1    0   CzVVJTZ4ssdqivqr1
2052     1    0  C0qgcP3QHZRIkuOkm8
2053     1    0  C50Is73ulQc4Xfpv5e
2055     1    0  Cqx3Nn1myl6TuT2YP6
3146     1    1   CeXTE32bb03yltkRV
3285     1    1  CJ0N5q2qFD0t1nCl3j


In [185]:
y_true = result_labels['true'].astype(int)
y_pred = result_labels['pred'].astype(int)

# pr_auc = average_precision_score(y_true, y_pred)

# roc_auc = roc_auc_score(y_true, y_pred)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

In [186]:
# print("PR AUC:", pr_auc)
# print("ROC AUC:", roc_auc)
print("Accuracy:", acc)
print("F1:", f1)
print(f"Precision: {precision}, Recall: {recall}")

Accuracy: 0.9966524650030432
F1: 0.35294117647058826
Precision: 1.0, Recall: 0.21428571428571427


## Additional Reference Code

In [4]:
# -*- coding: utf-8 -*-
# Markov Decision Process (MDP) - The Bellman equations adapted to
# Q Learning.Reinforcement Learning with the Q action-value(reward) function.
# Copyright 2019 Denis Rothman MIT License. See LICENSE.
import numpy as ql
# R is The Reward Matrix for each state
R = ql.matrix([ [0,0,0,0,1,0],
		            [0,0,0,1,0,1],
		            [0,0,100,1,0,0],
	             	[0,1,1,0,1,0],
		            [1,0,0,1,0,0],
		            [0,1,0,0,0,0] ])

# Q is the Learning Matrix in which rewards will be learned/stored
Q = ql.matrix(ql.zeros([6,6]))

"""##  The Learning rate or training penalty"""

# Gamma : It's a form of penalty or uncertainty for learning
# If the value is 1 , the rewards would be too high.
# This way the system knows it is learning.
gamma = 0.8

"""## Initial State"""

# agent_s_state. The agent the name of the system calculating
# s is the state the agent is going from and s' the state it's going to
# this state can be random or it can be chosen as long as the rest of the choices
# are not determined. Randomness is part of this stochastic process
agent_s_state = 5

"""## The random choice of the next state"""

# The possible "a" actions when the agent is in a given state
def possible_actions(state):
    current_state_row = R[state,]
    possible_act = ql.where(current_state_row >0)[1]
    return possible_act

# Get available actions in the current state
PossibleAction = possible_actions(agent_s_state)

# This function chooses at random which action to be performed within the range 
# of all the available actions.
def ActionChoice(available_actions_range):
    if(sum(PossibleAction)>0):
        next_action = int(ql.random.choice(PossibleAction,1))
    if(sum(PossibleAction)<=0):
        next_action = int(ql.random.choice(5,1))
    return next_action

# Sample next action to be performed
action = ActionChoice(PossibleAction)

"""## The Bellman Equation"""

# A version of the Bellman equation for reinforcement learning using the Q function
# This reinforcement algorithm is a memoryless process
# The transition function T from one state to another
# is not in the equation below.  T is done by the random choice above

def reward(current_state, action, gamma):
    Max_State = ql.where(Q[action,] == ql.max(Q[action,]))[1]

    if Max_State.shape[0] > 1:
        Max_State = int(ql.random.choice(Max_State, size = 1))
    else:
        Max_State = int(Max_State)
    MaxValue = Q[action, Max_State]
    
    # The Bellman MDP based Q function
    Q[current_state, action] = R[current_state, action] + gamma * MaxValue

# Rewarding Q matrix
reward(agent_s_state,action,gamma)

"""## Running the training episodes randomly"""

# Learning over n iterations depending on the convergence of the system
# A convergence function can replace the systematic repeating of the process
# by comparing the sum of the Q matrix to that of Q matrix n-1 in the
# previous episode
for i in range(50000):
    current_state = ql.random.randint(0, int(Q.shape[0]))
    PossibleAction = possible_actions(current_state)
    action = ActionChoice(PossibleAction)
    reward(current_state,action,gamma)
    
# Displaying Q before the norm of Q phase
print("Q  :")
print(Q)

# Norm of Q
print("Normed Q :")
print(Q/ql.max(Q)*100)

"""# Improving the program by introducing a decision-making process"""
nextc=-1
nextci=-1
conceptcode=["A","B","C","D","E","F"]
origin=int(input("index number origin(A=0,B=1,C=2,D=3,E=4,F=5): "))
print("Concept Path")
print("->",conceptcode[int(origin)])
for se in range(0,6):
    if(se==0):
        po=origin
    if(se>0):
        po=nextci
        #print("se:",se,"po:",po)
    for ci in range(0,6):
        maxc=Q[po,ci]
        #print(maxc,nextc)
        if(maxc>=nextc):
            nextc=maxc
            nextci=ci
            #print("next c",nextc)
    if(nextci==po):
        break;
    #print("present origin",po,"next c",nextci," ",nextc," ",conceptcode[int(nextci)])
    print("->",conceptcode[int(nextci)])


Q  :
[[  0.      0.      0.      0.    258.44    0.   ]
 [  0.      0.      0.    321.8     0.    207.752]
 [  0.      0.    500.    321.8     0.      0.   ]
 [  0.    258.44  401.      0.    258.44    0.   ]
 [207.752   0.      0.    321.8     0.      0.   ]
 [  0.    258.44    0.      0.      0.      0.   ]]
Normed Q :
[[  0.       0.       0.       0.      51.688    0.    ]
 [  0.       0.       0.      64.36     0.      41.5504]
 [  0.       0.     100.      64.36     0.       0.    ]
 [  0.      51.688   80.2      0.      51.688    0.    ]
 [ 41.5504   0.       0.      64.36     0.       0.    ]
 [  0.      51.688    0.       0.       0.       0.    ]]
Concept Path
-> A
-> E
-> D
-> C
